In [1]:
import numpy as np
from auxiliar_functions import TrainingData
import re
import os
from keras.layers import Dense
from keras.models import load_model, Sequential
from keras import backend as K
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from matplotlib import pyplot as plt
from functools import partial
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Função loss Customizada para magnitude
def mag_loss(y_pred, y_true, wall_val, xz_dict):
    """
        File: mag_isolated_prediction.py
        Function Name: mag_loss
        Summary: Função de custo para rede neural
        Description: Loss que adiciona a diferença da magnitude como penalidade
    """
    # Magnitude dos valores reais
    M_t = K.sqrt(K.sum(K.square(y_true), axis=-1))
    # Magnitude dos valores previstos
    M_p = K.sqrt(K.sum(K.square(y_pred), axis=-1))
    for indx, xz in enumerate(xz_dict):
        if xz_dict[indx] == wall_val[indx]:
            y_pred[indx] = tf.zeros(3)
            tf.Session().run(y_pred.eval())
        

    return K.mean(K.square(y_pred - y_true), axis=-1) + K.abs(M_p - M_t)


In [3]:
tf.reset_default_graph()

In [4]:
model = Sequential()
model.add(Dense(10, input_shape=(868,3), activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(3, activation='tanh'))

In [5]:
# Carregando dados para Treinamento
ANN_FOLDER = '/home/lucashqr/Documentos/Cursos/Keras Training/'\
             'Virtual/estudos-dissert/Keras_Virtual/Ciclone/ANN_DATA/'

DATA = TrainingData(ANN_FOLDER, scaler_dir='../../Models/Multi_Input/Scaler/')


# Dados de treinamento
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = DATA.data_gen(test_split=0.25)

Shape of X_TRAIN:  (77, 868, 3)
Shape of Y_TRAIN:  (77, 868, 3)
Shape of X_TEST:  (26, 868, 3)
Shape of Y_TEST:  (26, 868, 3)


In [6]:
xz = X_TRAIN[0, :, :2]
xz = DataFrame(xz)
print(xz.shape)
wall = []
for x,z in zip(xz[0], xz[1]):
    B = np.array([x,z]) - np.array([0.5, 0.5])
    if np.sqrt(np.sum(np.square(B))) >= 0.495:
        wall.append([x,z])
wall = np.array(wall) # pontos da parede 'mais afastados do centro'

# plt.plot(xz[0], xz[1], 'r*')
# plt.plot(wall[..., 0], wall[..., 1], 'b*')

(868, 2)


In [ ]:
TB = TensorBoard(log_dir='./test_loss/Mag_zero_wall4/')
# Interromper Treinamento
ES = EarlyStopping(monitor='loss', min_delta=0.000001, patience=175,
                   restore_best_weights=True, )

# Reduzir taxa de aprendizagem
RLRP = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=70, verbose=1,
                         min_lr=1E-10)


new_mag = partial(mag_loss, xz_dict=X_TRAIN[0,:2], wall_val=wall)

model.compile(optimizer='rmsprop', loss=new_mag)

model.fit(X_TRAIN, Y_TRAIN, epochs=30000, batch_size=4, callbacks=[TB, ES, RLRP])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


Epoch 1/30000
77/77 [==============================] - 0s 5ms/step - loss: 0.4216
Epoch 2/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.3164
Epoch 3/30000
 4/77 [>.............................] - ETA: 0s - loss: 0.2935

/home/lucashqr/.local/lib/python3.6/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/lucashqr/.local/lib/python3.6/site-packages/keras/callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `mae` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


77/77 [==============================] - 0s 2ms/step - loss: 0.3074
Epoch 4/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.3057
Epoch 5/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.3018
Epoch 6/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.2984
Epoch 7/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.2967
Epoch 8/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.2913
Epoch 9/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.2916
Epoch 10/30000
77/77 [==============================] - 0s 4ms/step - loss: 0.2891
Epoch 11/30000
77/77 [==============================] - 0s 3ms/step - loss: 0.2888
Epoch 12/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.2821
Epoch 13/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.2778
Epoch 14/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.2725
Epoch 15/30000
77/77 [===

77/77 [==============================] - 0s 2ms/step - loss: 0.1316
Epoch 199/30000
77/77 [==============================] - 0s 4ms/step - loss: 0.1348
Epoch 200/30000
77/77 [==============================] - 0s 4ms/step - loss: 0.1302
Epoch 201/30000
77/77 [==============================] - 0s 4ms/step - loss: 0.1323
Epoch 202/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.1312
Epoch 203/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.1319
Epoch 204/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.1314
Epoch 205/30000
77/77 [==============================] - 0s 3ms/step - loss: 0.1307A: 0s - loss: 0.
Epoch 206/30000
77/77 [==============================] - 0s 4ms/step - loss: 0.1300
Epoch 207/30000
77/77 [==============================] - 0s 4ms/step - loss: 0.1299
Epoch 208/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.1294
Epoch 209/30000
77/77 [==============================] - 0s 3ms/step - loss:

...

77/77 [==============================] - 0s 1ms/step - loss: 0.0323
Epoch 9614/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0324
Epoch 9615/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0339
Epoch 9616/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0325
Epoch 9617/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0335
Epoch 9618/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0332
Epoch 9619/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0330
Epoch 9620/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0326
Epoch 9621/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0334
Epoch 9622/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0327
Epoch 9623/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0326
Epoch 9624/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.03

77/77 [==============================] - 0s 2ms/step - loss: 0.0327
Epoch 9806/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0329
Epoch 9807/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0322
Epoch 9808/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0326
Epoch 9809/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0326
Epoch 9810/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0325
Epoch 9811/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0328
Epoch 9812/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0321
Epoch 9813/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0330
Epoch 9814/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0323
Epoch 9815/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0334
Epoch 9816/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.03

77/77 [==============================] - 0s 1ms/step - loss: 0.0323
Epoch 9998/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0331
Epoch 9999/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0313
Epoch 10000/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0329
Epoch 10001/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0331
Epoch 10002/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0323
Epoch 10003/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0327
Epoch 10004/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0328
Epoch 10005/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0326
Epoch 10006/30000
77/77 [==============================] - 0s 3ms/step - loss: 0.0330
Epoch 10007/30000
77/77 [==============================] - 0s 3ms/step - loss: 0.0327
Epoch 10008/30000
77/77 [==============================] - 0s 2ms/step - l

77/77 [==============================] - 0s 2ms/step - loss: 0.0326
Epoch 10188/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0327
Epoch 10189/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0320
Epoch 10190/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0324
Epoch 10191/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0323
Epoch 10192/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0325
Epoch 10193/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0325
Epoch 10194/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0336
Epoch 10195/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0316
Epoch 10196/30000
77/77 [==============================] - 0s 1ms/step - loss: 0.0324
Epoch 10197/30000
77/77 [==============================] - 0s 2ms/step - loss: 0.0325
Epoch 10198/30000
77/77 [==============================] - 0s 1ms/step -

77/77 [==============================] - 0s 2ms/step - loss: 0.0336
Epoch 10378/30000
68/77 [=========================>....] - ETA: 0s - loss: 0.0323

In [ ]:

# DF = [(read_csv(dado.path), re.findall(r'\d+\.?\d*_', dado.path)[0][:-1])
#               for dado in os.scandir(ANN_FOLDER)]

# XZ = np.array([np.array(sample) for sample in
#                        [dado[0][['Points:0', 'Points:2']] for dado in DF]])

#         # Valores de velocidade com o mesmo shape dos outros inputs
# INPUT_U = np.array([np.array(sample) for sample in
#                             [[float(dado[1])] * len(XZ[0]) for dado in DF]]).reshape(len(XZ), -1, 1)

# U_xyz = np.array([np.array(sample) for sample in
#                           [dado[0][['U:0', 'U:1', 'U:2']] for dado in DF]])

    

In [11]:
# Gerando dados para comparação com caso original
BASE_DIR ='./test_loss/Mag_zero_wall4/'
          
print(f"Gerando dados de previsão em {BASE_DIR}")
scaler_dict = DATA.return_scaler(load_sc=True)
VEL_ARR = np.array([[10.0]*868]).reshape(-1, 1)
VEL_ARR = scaler_dict['U_in'].transform(VEL_ARR).reshape(1, -1, 1)

# Valores previstos para Ux, Uy e Uz
PREDICs = model.predict(X_TRAIN[0, :].reshape(1, -1, 3))
print([p.shape for p in PREDICs])


# Retornando os dados para a escala anterior
Ux = DataFrame(scaler_dict['Ux_scaler'].inverse_transform(PREDICs[..., 0]).reshape(-1), columns=['U:0'])
Uy = DataFrame(scaler_dict['Uy_scaler'].inverse_transform(PREDICs[..., 1]).reshape(-1), columns=['U:1'])
Uz = DataFrame(scaler_dict['Uz_scaler'].inverse_transform(PREDICs[..., 2]).reshape(-1), columns=['U:2'])

# Inserindo valor dos pontos de Y
XYZ = read_csv(os.scandir(ANN_FOLDER).__next__().path)[['Points:0', 'Points:1', 'Points:2']]

# Geração de arquivo .CSV para leitura
FILENAME = f'NEW_SLICE_10_Isolated.csv'

SLICE_DATA = concat([Ux, Uy, Uz, XYZ], sort=True, axis=1)

# Escrevendo o header no formato do paraview
with open(BASE_DIR+FILENAME, 'w') as filename:
    HEADER = ''
    for col in list(SLICE_DATA.columns):
        HEADER += '\"' + col + '\",'
    filename.write(HEADER[:-1])
    filename.write('\n')

SLICE_DATA.to_csv(BASE_DIR + FILENAME, index=False, header=False, mode='a')
print("Dados de previsão copiados!")


# Diferença do valor previsto e o caso original
print("Calculando diferença...")
ORIGIN_DATA = read_csv(ANN_FOLDER+'SLICE_DATA_U_10_0.csv')

DIFF = SLICE_DATA[['U:0', 'U:1', 'U:2']] - ORIGIN_DATA[['U:0', 'U:1', 'U:2']]

RESULT_DATA = concat([DIFF, XYZ], axis=1)

print('Escrevendo dados DIFERENÇA')
RESULT_DATA.to_csv(BASE_DIR + 'DIFF_SLICE_U_10.csv', index=False)
print('Dados de diferença copiados!')


Gerando dados de previsão em ./test_loss/Mag_zero_wall4/
[(868, 3)]
Dados de previsão copiados!
Calculando diferença...
Escrevendo dados DIFERENÇA
Dados de diferença copiados!


In [13]:
model.save(BASE_DIR+'zero_wall_loss', overwrite=True, include_optimizer=True)

AttributeError: 'functools.partial' object has no attribute '__name__'